In [ ]:
%pip install snowflake-connector-python

In [ ]:
#Importing required libraries

import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.types import StringType
import pandas as pd
from dotenv import load_dotenv

In [ ]:
#Load Env variables
load_dotenv()

#Setup Credentials 
api_key = os.getenv('api_key')
account = os.getenv('account')
user = os.getenv('user')
password = os.getenv('password')
role = os.getenv('role')
warehouse = os.getenv('warehouse')
database = os.getenv('database')
schema = os.getenv('schema')
stage = os.getenv("stage")

In [ ]:
#Connecting to Snowflake

connection_parameters = {
"account": account,
"user": user,
"password": password,
"role": role,
"warehouse": warehouse,
"database": database,
"schema": schema
 }

#Creating a snowflake session
session = Session.builder.configs(connection_parameters).create() 

In [ ]:
#Defining Stored procedure logic within the function

@sproc(name="sproc_snowpark_demo", is_permanent=True, stage_location= stage, replace=True)
def sproc_snowpark_demo(session: Session) -> str:
    session.sql("""TRUNCATE TABLE TEMP_DB.PUBLIC.CUSTOMER_TABLE;""").collect()

    session.sql(""" INSERT INTO TEMP_DB.PUBLIC.CUSTOMER_TABLE
                            (
                                CUSTOMER_ID,         
                                DATE,                
                                SALUTATION,          
                                FIRST_NAME,          
                                LAST_NAME
                             )
                    SELECT C_CUSTOMER_SK
                                , TO_CHAR(D_DATE, 'YYYYMMDD')
                                , C_SALUTATION
                                , C_FIRST_NAME
                                , C_LAST_NAME
                    FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.CUSTOMER C
                    JOIN SNOWFLAKE_SAMPLE_DATA.TPCDS_SF100TCL.DATE_DIM D 
                    ON D_DATE_SK = C_FIRST_SHIPTO_DATE_SK
                    LIMIT 100
                    """).collect()

    result_df = session.sql("SELECT * FROM TEMP_DB.PUBLIC.CUSTOMER_TABLE").collect()

    return result_df

In [ ]:
#Calling the stored procedure

session.sql("""CALL sproc_snowpark_demo()
             """).show()